## How to run sourmash for viral genome taxonomy and recovery

How to run sourmash for viral taxonomic classifications?
What database works better, what k size, what scale? Does protein work?

## Steps
1. Simulate reads from the database file (either ICTV or genbank), using ngsngs
2a. create signature files directly from these reads
2b. Run reads through virome pipeline (assemly, Virsorter2)
3. predict proteins on predicted viruses
4. Create signatures
5. do fastmultigather (on 2a, 2b, 3) -> gather -> taxonomy
6. Genomad on 2b, compare to others


### Sequencing depth?
Roux et al, in silico benchmarking paper: Virome reads were simulated in silico with NeSSM (Jia et al., 2013) for each mock community (10,000,000 paired-end Illumina HiSeq reads, 2×100 bp).
wetlands paper: and paired-end 150 bp sequencing was done using the NovaSeq S4 platform (Illumina) to an approximate depth of 10 Gbp per virome. This is approx 35M paired end reads. 

So will do 2 things:
1x 150bp PE, 10M reads
1x 150bp PE, 35M reads


# Also want to test on a soil dataset (wetland data, from viromes):
https://datadryad.org/stash/dataset/doi:10.25338/B8C934

In [ ]:
# how to run a snakefile
srun --account=ctbrowngrp -p med2 -J simread -t 5:00:00 -c 6 --mem=30gb --pty bash
mamba activate branchwater
snakemake --use-conda --resources mem_mb=30000 --rerun-triggers mtime -c 6 --rerun-incomplete -k -n


In [ ]:
# I actually want to start with ICTV
# in: /home/ntpierce/2023-spillover/output.vmr/vmr_MSL38_v1.dna.fa.gz
# outputs in: /group/ctbrowngrp2/scratch/annie/2023-sourmash-viruses/ictv

In [ ]:
# use bbmap to split the fasta file 17 ways, so that each read file contains ~1000 sequences
partition.sh in=vmr_MSL38_v1.dna.fa out=split_vmr/vmr_%.fa ways=17

# sim reads using bbmap illumni novaseq 6000, => 75% of reads have quality score of over 30
randomreads.sh ref=../../resources/split_vmr/vmr_0.fa out1=vmr_0_R1.fq.gz out2=vmr_0_R2.fq.gz reads=10000000 simplenames=t minlength=150 \
maxlength=150 paired=t midq=30 


In [ ]:
# ngsngs 
# Using illumni novaseq 6000, => 75% of reads have quality score of over 30. Means 1:1000 bases is wrong 
# ngsngs  on all viral genomes, simulate 10000 reads, of length 150, PE, quality score of 35.

mamba activate ngsngs

#ngsngs in a loop
for ((i=1; i<=10; i++))
do
ngsngs -i vmr_MSL38_v1.dna.fa \
-r 10000000 \
-l 150 -seq PE -f fq.gz \
-o ./reads/vmrmsl38v1_sample_${i} -qs 35 -t 12
done

In [ ]:
# after virsorter2, have to individualize contigs to run prodigal
awk '/^>/ {OUT=substr($0,2) ".fa"}; OUT {print >OUT}' contigs.fa

# can use the individ contigs for the snakefile, aka running prodigal, sourmash. 
# rename contigs first. see snake

In [ ]:
# where are the ictv dbs:
# protein (k4-15, sc1)
/home/ntpierce/2023-spillover/output.vmr/vmr_MSL38_v1.protein.zip

# nucleotide (k15, sc1)
 /home/ntpierce/2023-spillover/output.vmr/vmr_MSL38_v1.dnak.zip
 # nucleotide (k21, sc1)
/home/ntpierce/2023-spillover/output.vmr/vmr_MSL38_v1.dna.zip

# taxonomy
/home/ntpierce/2023-spillover/output.vmr/vmr_MSL38_v1.taxonomy.csv

In [ ]:
# make a list of files for each of the signature files
readlink -f * > files.txt

# fastmultigather
srun --account=ctbrowngrp -p med2 -J mgmany -t 6:00:00 -c 30 --mem=30gb --pty bash
mamba activate branchwater

cd folder fmg

sourmash scripts fastmultigather \
../read.k10.tr \
../vmr_MSL38_v1.protein.zip -m protein \
-c 30 -k 10 -t 300 -s 100

sourmash scripts fastmultigather \
../read.dna.txt \
../vmr_MSL38_v1.dnak.zip \
-c 30 -k 15 -t 300 -s 100

In [ ]:
# get headers, put in snake
zcat file.fq.gz | grep '^@' > headers.txt

# get list of readfiles
readlink -f *.gz > ../all_simreads.txt

In [ ]:
# locations of the viral dbs for taxonomy:
# genbank, k=21,31 scaled=1
/home/ntpierce/2023-spillover/output.genbank-viral/genbank.2023-05.viral.dna.zip
# genbank, taxonomy
/home/ntpierce/database-releases/genbank-2023.05/viral.lineages.csv


# ICTV, k=21,31 scaled=1
/home/ntpierce/2023-spillover/output.vmr/vmr_MSL38_v1.dna.zip
# ICTV, taxonomy
/home/ntpierce/2023-spillover/output.vmr/vmr_MSL38_v1.taxonomy.csv

# Genbank
- For genbank there are a couple genomes that are present as fasta files but not in the tax file
- Therefore split those off, only keep fa files that have tax
- copy all genomes that are in the genbank database to local dir so we can make reads 
- Simulate reads, using either NeSSM or NGSNGS
- Create sourmash sig files from reads


In [ ]:
#  Copy the taxonomy file to current dir
cp ../../genbank-2023.05.viral.ncbi-lineages.csv /group/ctbrowngrp2/scratch/annie/2023-sourmash-viruses/genbank/

# then copy only genomes present in tax file to cur dir.
for f in $(cat only_name.txt)
do
cp /home/ntpierce/2023-spillover/genbank/genomes/${f%}_genomic.fna.gz ./fasta/
done

cat *.fna > ../genbank_viral.fa

number of viral contigs per sample
vmrmsl38v1_sample_1/final-viral-combined.fa:730
vmrmsl38v1_sample_10/final-viral-combined.fa:725
vmrmsl38v1_sample_2/final-viral-combined.fa:741
vmrmsl38v1_sample_3/final-viral-combined.fa:751
vmrmsl38v1_sample_4/final-viral-combined.fa:722
vmrmsl38v1_sample_5/final-viral-combined.fa:736
vmrmsl38v1_sample_6/final-viral-combined.fa:750
vmrmsl38v1_sample_7/final-viral-combined.fa:733
vmrmsl38v1_sample_8/final-viral-combined.fa:746
vmrmsl38v1_sample_9/final-viral-combined.fa:742